In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [ ]:
with sqlite3.connect('dataset/Cleaned.db') as conn:
    train = pd.read_sql_query('SELECT * FROM train_with_leaks', conn)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
labels = train.is_duplicate
y_true = list(map(int, labels))

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(train, y_true, stratify=y_true, test_size=0.3)

In [ ]:
Xtrain.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
from pyemd import emd
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/home/paperspace/w2v/GoogleNews-vectors-negative300.bin',
                                         binary=True)

In [ ]:
def sent2vec(sentence, model, method='tfidf', **kwargs):
    """
    Generic function to convert a sentence to a vector using
    avg or TFIDF vecorization.
    
    :param sentence: Sentence to be converted.
    :param model: The word2vec model
    """
    
    ##### It is recommended to pass seperate stopwords #####
    stopwords = kwargs.get('stopwords')
    if stopwords is None:
        from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
        stopwords = ENGLISH_STOP_WORDS
    
    ##### It is recommended to pass seperate tokenizers #####
    tokenizer = kwargs.get('tokenizer')
    if tokenizer is None:
        from nltk.tokenize import RegexpTokenizer
        tokenizer = RegexpTokenizer(r'\w+')

    words = tokenizer.tokenize(sentence) # Tokenize the words
    words = {each for each in words if each not in stopwords} # Remove all the stopwords
    
    V = []
    
    for word in words: # Process over all the words in the sentence
        if model.__contains__(word):
            V.append(model[word])
    V = np.array(V)
    
    # If no words were present in the model
    # or blank sentence was passed, return a
    # word vector with all 0's
    if V.shape[0] == 0:
        # If model returns word2vec of different size
        # Default value is taken 300
        custom_shape = kwargs.get('shape', 300)
        return np.zeros(custom_shape)
    
    # If there is atleast one word in the sentence that
    # was vectoried properly
    
    if method.lower() == 'avg':
        V = V.sum(axis=0)
        return V / np.sqrt((V ** 2).sum())
    
    elif method.lower() == 'tfidf':
        tfidf_model = kwargs.get('tfidf_model') # Load the tfidf model
        if tfidf_model: # If model loaded sucessfully
            tfidf_vec = tfidf_model.transform([sentence]) # get TFIDF for the sentence
            indx = tfidf_model.vocabulary_.get(word, -1)
            tfidfs = []
            for word in words:
                if model.__contains__(word):
                    if indx != -1:
                        tfidfs.append(tfidf_vec[0, indx])
                    else:
                        tfidfs.append(0.0)
            tfidfs = np.array(tfidfs)
            denominator = tfidfs.sum()
            if denominator == 0.0: # No word is representred in tfidf and w2v both
                # Better than skipping that sentence
                denominator = tfidf_model.idf_.min() * 0.01
            numerator = V * tfidfs.reshape(V.shape[0], 1)
            numerator = numerator.sum(axis=0)
            return numerator / denominator
        else:
            raise ValueError('No tfidf model is present')

In [ ]:
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
tfidf.fit(Xtrain.question1)

In [ ]:
question1_vectors_train = np.zeros((Xtrain.shape[0], 300))
for i, q in tqdm(enumerate(Xtrain.question1.values), total=283002):
    question1_vectors_train[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

In [ ]:
question1_vectors_test = np.zeros((Xtest.shape[0], 300))
for i, q in tqdm(enumerate(Xtest.question1.values), total=121287):
    question1_vectors_test[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

In [ ]:
tfidf.fit(Xtrain.question2)

In [ ]:
question2_vectors_train = np.zeros((Xtrain.shape[0], 300))
for i, q in tqdm(enumerate(Xtrain.question2.values), total=283002):
    question2_vectors_train[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

In [ ]:
question2_vectors_test = np.zeros((Xtest.shape[0], 300))
for i, q in tqdm(enumerate(Xtest.question2.values), total=121287):
    question2_vectors_test[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

In [ ]:
question1_vectors_train.shape

In [ ]:
question2_vectors_train.shape

In [ ]:
Xtrain.drop([
    'id',
    'question1',
    'question2',
    'is_duplicate'
], axis=1, inplace=True)

In [ ]:
Xtest.drop([
    'id',
    'question1',
    'question2',
    'is_duplicate'
], axis=1, inplace=True)

In [ ]:
X_train = np.hstack((question1_vectors_train, question2_vectors_train, np.array(Xtrain)))
X_test = np.hstack((question1_vectors_test, question2_vectors_test, np.array(Xtest)))

In [ ]:
import xgboost as xgb
from sklearn.metrics import log_loss

In [ ]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.1
params['max_depth'] = 5
params['silent'] = 1
params['subsample'] = 1
params['colsample_bytree'] = 0.3
params['reg_alpha'] = 100
params['reg_lambda'] = 100

d_train = xgb.DMatrix(X_train, label=ytrain)
d_test = xgb.DMatrix(X_test, label=ytest)

watchlist = [(d_train, 'train'), (d_test, 'valid')]

clf = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=1)
predict_y = clf.predict(d_test)
print('Test Log-Loss: ', log_loss(ytest, predict_y, eps=1e-15))

**LOGLOSS**: 0.3204

As we can see that there is almost 0.03 improvement with this Leaky feature. But I guess the model is still slightly overfitting but how much it affects the leaderboard, that's the bigger question